# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [13]:
from torch import optim

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1,16,5),
                                    nn.BatchNorm2d(16),
                                    nn.ReLU(inplace=True),
                                    nn.MaxPool2d(2,2))
        self.layer2 = nn.Sequential(nn.Conv2d(16,32,5),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(inplace=True),
                                    nn.MaxPool2d(2,2))
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(nn.Linear(32*4*4,64),
                                nn.ReLU(inplace=True),
                                nn.Linear(64,10))
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self,x): # (,1,28,28)
        out = self.layer1(x) # (,16,12,12)
        out = self.layer2(out) # (,32,4,4)
        out = self.flatten(out)
        out = self.fc(out)
        out = self.softmax(out)
        return out
    
    
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model = SimpleNet().to(device)

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

Using cuda device


Next, we can start to train and evaluate!

In [14]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    running_loss, running_acc = 0., 0.
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images, labels = images.to(device), labels.to(device)
        out = model(images)
        loss = criterion(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
        running_loss += loss.item() * labels.size(0)
        pred = out.argmax(dim=1)
        running_acc += (pred == labels).sum()
        
    print('Epoch {} : Loss = {:.6f}, Acc = {:.6f}'.format(epoch+1,
                                                          running_loss / len(train_dataset),
                                                          running_acc / len(train_dataset)))
    
acc = 0
for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)
    out = model(images)
    pred = out.argmax(dim=1)
    acc += (pred == labels).sum()

print('Test Acc: {:.6f}'.format(acc / len(test_dataset)))

  1%|          | 4/468 [00:00<00:12, 38.47it/s]

Epoch 1 : Loss = 1.534699, Acc = 0.943033


  1%|          | 4/468 [00:00<00:12, 37.64it/s]

Epoch 2 : Loss = 1.477865, Acc = 0.982367


  1%|          | 4/468 [00:00<00:12, 38.23it/s]

...

Epoch 9 : Loss = 1.466169, Acc = 0.991700


 10%|█         | 8/78 [00:00<00:00, 73.34it/s]

Epoch 10 : Loss = 1.465662, Acc = 0.992200


100%|██████████| 78/78 [00:01<00:00, 72.20it/s]

Test Acc: 0.990300


#### Q5:
Please print the training and testing accuracy.

Printed above.